In this notebook, I'm going to attempt to run all the techniques on the same circuit

In [1]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits, ghz_circuits
from mitiq import MeasurementResult, Observable, PauliString, raw
from mitiq.shadows.shadows import *
from mitiq.shadows.shadows_utils import *

In [ ]:
n_qubits = 2

In [ ]:
circuit = generate_rb_circuits(2, 10)[0]

In [4]:
def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))

    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

In [5]:
from mitiq import zne


true_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit)
zne_value = zne.execute_with_zne(circuit, execute)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w ZNE:    {abs((true_value - zne_value) / true_value):.3f}")

Error w/o  Mitiq: 0.378
Error w Mitiq:    0.031


In [11]:
# DDD
from mitiq import ddd

rule = ddd.rules.xyxy # low key I have no idea what this means

ddd_value = ddd.execute_with_ddd(
    circuit,
    execute,
    rule=rule,
)
print(f"Error w DDD:    {abs((true_value - ddd_value) / true_value):.3f}")

Error w DDD:    0.378


In [13]:
# QSE
from mitiq import qse

Ms = [ # chat GPT generated stabilizer group
    "II",
    "XI",
    "ZI",
    "IX",
    "IZ"
]
check_operators = [
    PauliString(M, coeff=1, support=range(n_qubits)) for M in Ms
] # low key copy pasta-ed these so they may not even be the right choice
negative_Ms_as_pauliStrings = [
    PauliString(M, coeff=-1, support=range(n_qubits)) for M in Ms
] # low key copy pasta-ed these so they may not even be the right choice
code_hamiltonian = Observable(*negative_Ms_as_pauliStrings)

qse.execute_with_qse(
    circuit,
    execute,
    check_operators, # not sure what these are, need further reading
    code_hamiltonian,
    observable=None
)
negative_Ms_as_pauliStrings = [
    PauliString(M, coeff=-1, support=range(n_qubits)) for M in Ms
] # low key copy pasta-ed these so they may not even be the right choice
code_hamiltonian = Observable(*negative_Ms_as_pauliStrings)

qse.execute_with_qse(
    circuit,
    execute,
    check_operators, # not sure what these are, need further reading
    code_hamiltonian,
    observable=None
)

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None

In [33]:
# CDR
from mitiq import cdr

cdr_expval = cdr.execute_with_cdr(
    circuit,
    execute,
    observable=obs,
    simulator=partial(execute, noise_level=0, p0=0),
    seed=0,
).real
print(f"mitigated expectation value with CDR: {cdr_expval:.2f}")

mitigated expectation value with CDR: 2.00


In [37]:
# # Pauli Twirling
# from mitiq import pt

# PT_circuit = pt.pauli_twirl_circuit(
#     circuit=circuit, # wait wtf but the noise isn't tied to the circuit???
# )
# # and wtf do I execute
# print(f"mitigated expectation value with Pauli Twirling: {PT_result:.2f}")

AttributeError: 'list' object has no attribute 'with_noise'

In [79]:
# DDD + ZNE
ddd_executor = ddd.mitigate_executor(execute, rule=rule)

ddd_zne_executor = zne.mitigate_executor(ddd_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

ddd_zne_result = ddd_zne_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(ddd_zne_result.real))

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None

In [81]:
# ZNE + DDD
zne_executor = zne.mitigate_executor(execute, scale_noise=zne.scaling
.folding.fold_global)

zne_ddd_executor = ddd.mitigate_executor(zne_executor, observable=obs, rule=rule)

zne_ddd_result = zne_ddd_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(zne_ddd_result.real))

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None